In [ ]:
# plotly_dashboard_final.py
# Plotly tabanlı interaktif dashboard (treemap fix + robust temizlik)

from pathlib import Path
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

DATA_EMP = Path("employee_master.csv")
DATA_LOG = Path("leave_absence_log.csv")
OUT_DIR = Path("charts_html")
OUT_DIR.mkdir(exist_ok=True)

# ---------- CSV yoksa mini sentetik veri üret ----------
def ensure_datasets():
    if DATA_EMP.exists() and DATA_LOG.exists():
        return
    rng = np.random.default_rng(42)
    # çalışanlar
    emp = pd.DataFrame({
        "employee_id": [f"E{i:04d}" for i in range(1,61)],
        "department": rng.choice(
            ["Sales","Marketing","HR","Finance","IT","Operations"], size=60)
    })
    # izin kayıtları
    recs = []
    leave_types = ["Annual","Sick","Unpaid","Personal"]
    for eid in emp["employee_id"]:
        n = int(rng.poisson(6)) + 1
        for _ in range(n):
            lt = rng.choice(leave_types, p=[0.5,0.35,0.1,0.05])
            y  = int(rng.choice([2024, 2025], p=[0.6, 0.4]))
            m  = int(rng.integers(1, 13))
            d  = int(rng.integers(1, 28))
            dur = int(rng.integers(1, 8))
            start = pd.Timestamp(y, m, d)
            recs.append({
                "employee_id": eid,
                "leave_type": lt,
                "start_date": start,
                "duration_days": dur
            })
    log = pd.DataFrame(recs)
    emp.to_csv(DATA_EMP, index=False)
    log.to_csv(DATA_LOG, index=False)

ensure_datasets()

# ---------- Load ----------
log = pd.read_csv(DATA_LOG, parse_dates=["start_date"])
emp = pd.read_csv(DATA_EMP)

# zorunlu kolon kontrolleri
needed_cols = {"employee_id","leave_type","start_date","duration_days"}
missing = needed_cols - set(log.columns)
if missing:
    raise ValueError(f"leave_absence_log.csv içinde şu kolonlar eksik: {missing}")

# ---------- Hazırlık & Temizlik ----------
# sayısallaştırma (treemap için kritik)
log["duration_days"] = pd.to_numeric(log["duration_days"], errors="coerce")

# tarih türevleri
log["year"] = log["start_date"].dt.year
log["year_month"] = log["start_date"].dt.to_period("M").astype(str)

def leave_type_counts(df: pd.DataFrame) -> pd.DataFrame:
    return (df["leave_type"]
              .value_counts(dropna=False)
              .rename_axis("leave_type")
              .reset_index(name="count"))

# ---------- 1) İzin türü bar grafiği + Yıl filtresi ----------
years = sorted([int(y) for y in log["year"].dropna().unique().tolist()])
fig1 = go.Figure()

# All
base_counts = leave_type_counts(log)
fig1.add_bar(x=base_counts["leave_type"], y=base_counts["count"], name="All")

# Year-specific (başta görünmez)
for y in years:
    c = leave_type_counts(log[log["year"] == y])
    fig1.add_bar(x=c["leave_type"], y=c["count"], name=str(y), visible=False)

buttons = []
buttons.append(dict(
    label="All",
    method="update",
    args=[{"visible": [True] + [False]*len(years)},
          {"title": "İzin Türleri (Tüm Yıllar)"}]
))
for i, y in enumerate(years, start=1):
    vis = [False]*(len(years)+1)
    vis[i] = True
    buttons.append(dict(
        label=str(y),
        method="update",
        args=[{"visible": vis}, {"title": f"İzin Türleri ({y})"}]
    ))

fig1.update_layout(
    title="İzin Türlerine Göre Kayıt Dağılımı",
    xaxis_title="İzin Türü",
    yaxis_title="Kayıt Sayısı",
    updatemenus=[dict(active=0, buttons=buttons, x=1.05, xanchor="left", y=1.15)]
)
fig1.write_html(str(OUT_DIR / "izin_turleri.html"), include_plotlyjs="cdn")

# ---------- 2) Aylık toplam gün (range slider) ----------
monthly = (log.groupby("year_month")["duration_days"]
             .sum(min_count=1)  # eğer hep NaN ise toplam NaN olmasın
             .fillna(0)
             .reset_index(name="total_days"))
fig2 = px.line(monthly, x="year_month", y="total_days",
               markers=True, title="Aylık Toplam İzin Günleri")
fig2.update_layout(xaxis=dict(rangeslider=dict(visible=True)))
fig2.update_traces(hovertemplate="Ay: %{x}<br>Toplam gün: %{y}")
fig2.write_html(str(OUT_DIR / "aylik_toplam.html"), include_plotlyjs="cdn")

# ---------- 3) Departman x Ay ısı haritası ----------
dept_counts = (log.merge(emp[["employee_id","department"]], on="employee_id", how="left")
                 .dropna(subset=["department"])
                 .groupby(["department","year_month"])["duration_days"]
                 .sum(min_count=1).fillna(0).reset_index())
heat = dept_counts.pivot(index="department", columns="year_month", values="duration_days").fillna(0)
fig3 = px.imshow(
    heat,
    labels=dict(color="Gün"),
    title="Departman x Ay Isı Haritası (Toplam İzin Günleri)",
    aspect="auto"
)
fig3.write_html(str(OUT_DIR / "dept_heatmap.html"), include_plotlyjs="cdn")

# ---------- 4) Treemap: Dept -> Leave Type (FIXLİ) ----------
tree_df = (log.merge(emp[["employee_id","department"]], on="employee_id", how="left")
             .dropna(subset=["department","leave_type"])
             .groupby(["department","leave_type"])["duration_days"]
             .sum(min_count=1)
             .reset_index())

# sayısallaştır ve filtrele
tree_df["duration_days"] = pd.to_numeric(tree_df["duration_days"], errors="coerce").fillna(0)
tree_df = tree_df[tree_df["duration_days"] > 0]

# eğer hâlâ boşsa; en azından “kayıt yok” mesajı için dummy veri ekle
if tree_df.empty:
    tree_df = pd.DataFrame({
        "department": ["No Data"],
        "leave_type": ["-"],
        "duration_days": [1]
    })

fig4 = px.treemap(
    tree_df,
    path=["department","leave_type"],
    values="duration_days",
    title="Departman → İzin Türü (Toplam Gün) Treemap"
)
fig4.write_html(str(OUT_DIR / "treemap.html"), include_plotlyjs="cdn")

# ---------- 5) Leaderboard tablo ----------
top_emp = (log.groupby("employee_id")["duration_days"].sum(min_count=1)
             .fillna(0)
             .reset_index(name="total_days")
             .sort_values("total_days", ascending=False)
             .head(20)
             .merge(emp[["employee_id","department"]], on="employee_id", how="left"))
fig5 = go.Figure(data=[go.Table(
    header=dict(values=["Employee ID","Department","Total Days"],
                fill_color="lightgrey", align="left"),
    cells=dict(values=[top_emp["employee_id"], top_emp["department"], top_emp["total_days"]],
               align="left")
)])
fig5.update_layout(title="Top 20 Çalışan (Toplam İzin Günleri)")
fig5.write_html(str(OUT_DIR / "leaderboard.html"), include_plotlyjs="cdn")

# ---------- 6) Tek sayfa dashboard ----------
parts = []
for f in ["izin_turleri.html","aylik_toplam.html","dept_heatmap.html","treemap.html","leaderboard.html"]:
    parts.append(f'<iframe src="{f}" style="width:100%;height:520px;border:none;"></iframe>')

dashboard_html = f"""<!doctype html>
<html>
<head>
<meta charset="utf-8" />
<title>Leave & Absence Dashboard</title>
<style>
 body {{ font-family: Arial, sans-serif; margin: 0; padding: 20px; background:#0b0c10; color:#eee; }}
 h1 {{ margin: 0 0 10px; }}
 .grid {{ display: grid; grid-template-columns: 1fr; gap: 20px; }}
 @media (min-width: 1000px) {{ .grid {{ grid-template-columns: 1fr 1fr; }} }}
 iframe {{ background: #fff; border-radius: 10px; box-shadow: 0 8px 24px rgba(0,0,0,.25); }}
 a.btn {{ display:inline-block; margin:8px 0 16px; padding:8px 12px; background:#1f2833; color:#66fcf1; text-decoration:none; border-radius:8px; }}
</style>
</head>
<body>
  <h1>Çalışan İzin & Devamsızlık — Interaktif Dashboard</h1>
  <a class="btn" href="izin_turleri.html" target="_blank">İzin Türleri</a>
  <a class="btn" href="aylik_toplam.html" target="_blank">Aylık Trend</a>
  <a class="btn" href="dept_heatmap.html" target="_blank">Departman Isı</a>
  <a class="btn" href="treemap.html" target="_blank">Treemap</a>
  <a class="btn" href="leaderboard.html" target="_blank">Leaderboard</a>
  <div class="grid">
    {''.join(parts)}
  </div>
</body>
</html>
"""
(OUT_DIR / "dashboard.html").write_text(dashboard_html, encoding="utf-8")
print("Hazır: charts_html/dashboard.html")


Hazır: charts_html/dashboard.html


In [ ]:
import shutil
from google.colab import files
shutil.make_archive("charts_html", 'zip', "charts_html")
files.download("charts_html.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>